# NASA demo template

In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML('<style>.custom-slider .bk-input-group {height: 400px;}</style>'))
display(HTML('<style>.small-custom-slider .bk-input-group {height: 200px;}</style>'))

#slider = pnw.FloatSlider(start=-0.5, end=0.5, value=0,  orientation='vertical', css_classes=["custom-slider"] )

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gs
from matplotlib.animation import FuncAnimation
import concurrent.futures
from IPython.display import clear_output
from IPython.utils import io
# from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import *

In [3]:
%%capture
from OpenVisus import *

In [4]:
%%capture
colormaps = ['viridis', 'plasma', 'inferno', 'magma', 'cividis','ocean', 'gist_earth', 'terrain', 'gist_stern',
             'gnuplot', 'gnuplot2', 'CMRmap', 'cubehelix', 'brg',
             'gist_rainbow', 'rainbow', 'jet', 'turbo', 'nipy_spectral',
             'gist_ncar']

In [5]:
%%capture
def Assert(cond):
    if not cond:
        raise Exception("Assert failed")

class CachedDataset(PyDataset):
    
    # constructor
    def __init__(self, args):
        self.local_filename=os.path.abspath(args["local"]).replace("\\","/")
        self.remote_url=args["url"]
        self.remote_access_type = args["access"]
        self.description=args["description"]
        
        super().__init__(LoadDatasetCpp(self.remote_url))
        
        self.num_blocks = len(self.getFields()) * self.getTotalNumberOfBlocks() * len(self.getTimesteps().asVector())
        self.num_blocks_cached = 0

        self.stop_thread=False
        self.thread=None
        
        self.progress=None
        self.progress_display=None
        
    def __del__(self):
        self.stopCaching()   
        
    # createAccess
    def createAccess(self, ):
        
        access_config="""
            <access type='multiplex'>
                    <access type='disk' chmod='rw' url='file://{}' />
                    <access type='{}' url='{}' chmod="r" /> 
            </access>  
        """.format(
            self.local_filename.replace("&","&amp;"),
            self.remote_access_type,
            self.remote_url.replace("&","&amp;")) 
        
        # print("Creating access",access_config)

        access= self.createAccessForBlockQuery(StringTree.fromString(access_config));

        # at this point the cache is enabled with the new local idx file
        Assert(os.path.isfile(self.local_filename));

        return access   

    # startCaching
    def startCaching(self, background=True):
        
        if background:
            self.thread = threading.Thread(target=self.startCaching, args=(False,));
            self.stop_thread=False
            self.thread.start()        
            return 

        #print("start caching","...")
        
        access=self.createAccess()

        access.beginRead()
        
        for field in self.getFields():
            for blockid in range(self.getTotalNumberOfBlocks()): 
                for time in self.getTimesteps().asVector():
                    # print("Copying block","time",time,"field",field,"blockid",blockid,"...")
                    buffer =  self.readBlock(blockid, field=field, time=time, access=access);
                    
                     # to debug missing blocks
                    if  False and buffer is None :
                        read_block = db.createBlockQuery(blockid, ord('r'));
                        msg="# {} {} \n".format(blockid,read_block.getLogicBox().toString());
                        os.write(1, bytes(msg,'utf-8'))                   
                    
                    self.num_blocks_cached += 1
                    self.updateProgress()
                    if self.stop_thread:
                        # print("thread stopped")
                        access.endRead()
                        return
                        
        access.endRead()
        self.thread=None
        #print("caching finished done")
        
    # stopCaching
    def stopCaching(self):
        #print("stopping caching...")
        self.stop_thread=True
        if self.thread:
            self.thread.join()
            self.thread=None
    # getWidth
    def getWidth(self):
        p2=self.getLogicBox()[1]
        return p2[0]    
        
    # getHeight
    def getHeight(self):
        p2=self.getLogicBox()[1]
        return p2[1]   
        
    # getDepth
    def getDepth(self):
        p2=self.getLogicBox()[1]
        return p2[2]  
        
    # readSlice
    def readSlice(self,dir=0, slice=0,quality=-3, time=1, access=None):
        
        W,H,D=self.getWidth(), self.getHeight(), self.getDepth()
        x=[0,W] if dir!=0 else [slice,slice+1]
        y=[0,H] if dir!=1 else [slice,slice+1]
        z=[0,D] if dir!=2 else [slice,slice+1] 
        ret=self.read(x=x, y=y,z=z, quality=quality,time=time,access=access)
        
        width,height=[value for value in ret.shape if value>1]
        return ret.reshape([width,height])
        
    # readColumn
    def readXYColumn(self,Height, Depth,quality=-3, time=1, access=None):
        W,H,D=self.getWidth(), self.getHeight(), self.getDepth()
        x=[0,W]
        y=[Height,Height+1]
        z=[Depth ,Depth +1] 
        ret=self.read(x=x, y=y,z=z, quality=quality,time=time,access=access)
        #print(">",ret.shape)
        width=[value for value in ret.shape if value>1]
        return ret
        
    # setProgress
    def setProgress(self,progress, progress_display):
        self.progress=progress
        self.progress_display=progress_display   
        self.progress.min=0
        self.progress.max =self.num_blocks       

    # updateProgress
    def updateProgress(self):
                    
        if self.progress:
            self.progress.value = self.num_blocks_cached

        if self.progress_display:
            self.progress_display.value = (
                "Caching progress %.2f%% (%d/%d)" % (
                    100 * self.num_blocks_cached/self.num_blocks, 
                    self.num_blocks_cached,
                    self.num_blocks))                    

print("Utilities defined")

In [6]:
# decorater used to block function printing to the console
def blockPrinting(func):
    def func_wrapper(*args, **kwargs):
        # block all printing to the console
        sys.stdout = open(os.devnull, 'w')
        # call the method in question
        value = func(*args, **kwargs)
        # enable all printing to the console
        sys.stdout = sys.__stdout__
        # pass the return value of the method back
        return value

    return func_wrapper

In [7]:
NasaAtmosphericZone = []

for i in range(6):
    datasetName = "nasa-DYAMOND-atmospheric-face_"+str(i)+"_depth_52_time_1024"
    NasaAtmosphericlocal_cache="./visus-cache/"+datasetName+"/visus.idx"
    NasaAtmospheric =    {
            "url":"http://atlantis.sci.utah.edu/mod_visus?dataset="+datasetName+"&cached=1",
            "access":"network",
            "local": NasaAtmosphericlocal_cache,
            "description":'University of Utah Campus Server'
        }



    NasaAtmosphericdb=CachedDataset(NasaAtmospheric )
    NasaAtmosphericaccess=NasaAtmosphericdb.createAccess()

    NasaAtmosphericZone.append([NasaAtmosphericdb, NasaAtmosphericaccess])



In [8]:
@blockPrinting
def getHorizontalImage(depth,time,data,x): 
    global NasaAtmosphericZone
    db =NasaAtmosphericZone[x][0] 
    access=NasaAtmosphericZone[x][1]
    return db.readSlice(dir=2, slice=(depth//2)*2,access=access,time=time, quality=-5)




def getPlot(data,vmin,vmax):
    plt.tick_params(axis='x', which='both', bottom=False,
                top=False, labelbottom=False)
    plt.tick_params(axis='y', which='both', right=False,
                left=False, labelleft=False)
    for pos in ['right', 'top', 'bottom', 'left']:
        plt.gca().spines[pos].set_visible(False)
    plt.imshow(data, origin = 'lower', cmap ='coolwarm',   aspect='auto',  vmin = vmin, vmax = vmax)
    
    return
    

In [9]:

def getNormData(time,depth):
    clear_output(wait=True)
    with concurrent.futures.ThreadPoolExecutor() as executor:
        a=executor.submit(getHorizontalImage,depth,time,NasaAtmosphericZone,x=0)
        a1=executor.submit(getHorizontalImage,depth,time,NasaAtmosphericZone,x=1)
        a2=executor.submit(getHorizontalImage,depth,time,NasaAtmosphericZone,x=2)
        a3=executor.submit(getHorizontalImage,depth,time,NasaAtmosphericZone,x=3)
        a4=executor.submit(getHorizontalImage,depth,time,NasaAtmosphericZone,x=4)
        a5=executor.submit(getHorizontalImage,depth,time,NasaAtmosphericZone,x=5)
        data=a.result()
        data1=a1.result()
        data2=a2.result()
        data3=a3.result()
        data4=a4.result()
        data5=a5.result()
    
    data3=np.rot90(data3)
    data4=np.rot90(data4)
    data5=np.rot90(np.rot90(np.rot90(data5)))
    clear_output(wait=True)
    minn=[]
    maxx=[]
    for i in range(0,6):
        minn.append(np.min(getHorizontalImage(depth,1,NasaAtmosphericZone,x=i )))
        maxx.append(np.max(getHorizontalImage(depth,1,NasaAtmosphericZone,x=i )))
    vmin=np.min(minn)
    vmax=np.max(maxx)
    
    
    fig = plt.figure(figsize = (200, 100),frameon=False)
    

    gs1=plt.GridSpec(nrows=3,ncols=4)
    plt.subplots_adjust(left=0, bottom=0, right=0.1, top=0.1, wspace=0, hspace=0)


    plt.subplot(gs1[1,0])
    getPlot(data,vmin,vmax)
     
    plt.subplot(gs1[1,1])
    getPlot(data1,vmin,vmax)
     
    plt.subplot(gs1[0,1])
    getPlot(data2,vmin,vmax)
     
    plt.subplot(gs1[1,2])
    getPlot(data3,vmin,vmax)
    plt.subplot(gs1[1,3])
    getPlot(data4,vmin,vmax)
           
    plt.subplot(gs1[2,1])
    getPlot(data5,vmin,vmax)
    plt.show()
    return

 

In [10]:
time=widgets.IntSlider(min=1,max=1024,step=1,value=0,layout=Layout(width="500px")) 
depth=widgets.IntSlider(min=1,max=51,step=1,value=51,layout=Layout(width="500px"))
show_data=interactive(getNormData,time=time,depth=depth)

In [11]:
show_data

interactive(children=(IntSlider(value=1, description='time', layout=Layout(width='500px'), max=1024, min=1), I…